This is to create a broucher to the company which can be used to clients,investors and potential recruits

In [2]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
client = OpenAI()

API key looks good so far


In [4]:
links=fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
links_system_prompt=f"""
you are provided with the list of links found in the webpage.
you are able to decide which of the links would be most relevant to include in the broucher about the company,
such as lins to About page, or Company page, or Career/Jobs pages.
you should respond in the json sample as below example:
{{
    "links": [
        {{"type": "about page", "url": "https://full.url/goes/here/about"}},
        {{"type": "careers page", "url": "https://another.full.url/careers"}}
    ]
}}
"""

In [11]:
def get_links_user_prompt(url):
    user_prompt=f"""
    Here is the list of links on the website {url} -
    Please decide which of these are relevant web links for a brochure about the company, 
    respond with the full https URL in JSON format.
    Do not include Terms of Service, Privacy, email links.

    Links (some might be relative links):
    
    """
    links=fetch_website_links(url)
    user_prompt+="\n".join(links)
    return user_prompt


In [12]:
print(get_links_user_prompt("https://edwarddonner.com"))


    Here is the list of links on the website https://edwarddonner.com -
    Please decide which of these are relevant web links for a brochure about the company, 
    respond with the full https URL in JSON format.
    Do not include Terms of Service, Privacy, email links.

    Links (some might be relative links):

    https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic

In [15]:
def select_relevant_links(url):
    response=client.chat.completions.create(model=MODEL,messages=[
        {"role":"system","content":links_system_prompt},
        {"role":"user","content":get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    results=response.choices[0].message.content
    links=json.loads(results)
    return links

In [16]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'},
  {'type': 'nebula page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'}]}

In [44]:
# Improved version with timeout and error handling
def select_relevant_links(url):
    print(f"Calling {MODEL} to select relevant links for {url}...")
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role":"system","content":links_system_prompt},
                {"role":"user","content":get_links_user_prompt(url)}
            ],
            response_format={"type": "json_object"},
            timeout=60  # Add timeout to prevent hanging
        )
        results = response.choices[0].message.content
        print("Response received, parsing JSON...")
        links = json.loads(results)
        print(f"Successfully parsed {len(links.get('links', []))} relevant links")
        return links
    except Exception as e:
        print(f"Error occurred: {type(e).__name__}: {e}")
        raise


In [17]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

#Make the Broucher

In [22]:
def fetch_page_and_all_relevent_links(url):
    content=fetch_website_contents(url)
    relavent_links=select_relevant_links(url)
    result = f"## Landing Page:\n\n {content}\n## Relavent Links :\n"
    for link in relavent_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result


In [23]:
print(fetch_page_and_all_relevent_links("https://huggingface.co"))

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


## Landing Page:

 Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Tongyi-MAI/Z-Image-Turbo
Updated
6 days ago
•
278k
•
2.69k
microsoft/VibeVoice-Realtime-0.5B
Updated
2 days ago
•
131k
•
817
zai-org/GLM-4.6V-Flash
Updated
5 days ago
•
67.7k
•
409
mistralai/Devstral-Small-2-24B-Instruct-2512
Updated
40 minutes ago
•
15.8k
•
325
zai-org/GLM-4.6V
Updated
5 days ago
•
3.37k
•
297
Browse 1M+ models
Spaces
Running
on
Zero
553
Z Image Turbo
🖼
553
Generate stunning images from text prompts
Running
on
Zero
MCP
Featured
1.37k
Z Image Turbo
🏃
1.37k
Generate images from text prompts
Running
on
Zero
MCP
126
Qwen Image to LoRA
😭
126
Generate LoRA from a single image
Running
on
Zero
Featured
947
Dream-wan2-2

In [25]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [ ]:


def get_brochure_user_prompt(company_name,url):
    user_prompt=f"""
    You are looking at a company called: {company_name}
    Here are the contents of its landing page and other relevant pages;
    use this information to build a short brochure of the company in markdown without code blocks.\n\n
    """
    user_prompt += fetch_page_and_all_relevent_links(url)
    user_prompt = user_prompt[:5_000] # truncate if morethan 5000 characters
    return user_prompt



In [30]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

"\n    You are looking at a company called: HuggingFace\n    Here are the contents of its landing page and other relevant pages;\n    use this information to build a short brochure of the company in markdown without code blocks.\n\n\n    ## Landing Page:\n\n Hugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nTongyi-MAI/Z-Image-Turbo\nUpdated\n6 days ago\n•\n278k\n•\n2.69k\nmicrosoft/VibeVoice-Realtime-0.5B\nUpdated\n2 days ago\n•\n131k\n•\n819\nzai-org/GLM-4.6V-Flash\nUpdated\n5 days ago\n•\n67.7k\n•\n410\nmistralai/Devstral-Small-2-24B-Instruct-2512\nUpdated\nabout 1 hour ago\n•\n15.8k\n•\n326\nzai-org/GLM-4.6V\nUpdated\n5 days ago\n•\n3.37k\n•\n297\nBrowse 1M+ models\nSpaces\

In [34]:
def create_brochure(company_name,url):
    response=client.chat.completions.create(model="gpt-4.1-nano",messages=[{
        "role":"system","content":brochure_system_prompt
    },
    {"role":"user","content":get_brochure_user_prompt(company_name,url)}])
    result=response.choices[0].message.content
    display(Markdown(result))

In [35]:
create_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face: Pioneering the Future of AI

## About the Company
Hugging Face is at the forefront of the AI revolution, creating a vibrant community centered around machine learning innovation. As a collaborative platform, it enables researchers, developers, and organizations to share, discover, and develop state-of-the-art models, datasets, and applications. The company fosters an open and ethical approach to AI development, positioning itself as the hub for the next generation of AI solutions.

## What We Do
- **Models & Datasets:** Explore over 1 million open-source machine learning models and datasets across diverse modalities—including text, images, videos, audio, and 3D.
- **Spaces:** Host and showcase live demos and applications built on advanced AI models, ranging from image generation to speech synthesis.
- **Community & Collaboration:** Connect with a global network of AI enthusiasts, researchers, and companies to collaborate on cutting-edge projects.
- **Enterprise Solutions:** Accelerate your organization's AI initiatives with paid compute resources and enterprise-grade products designed for scalable, secure, and efficient AI development.

## Our Platform
Hugging Face provides an open-source stack that accelerates innovation and reduces barriers in AI development. Whether you're building a portfolio, contributing to open-source ML projects, or deploying enterprise solutions, our platform offers tools and infrastructure to support your journey.

## Company Culture & Values
- **Open & Collaborative:** We champion transparency, community-driven development, and shared knowledge to democratize AI.
- **Innovative Spirit:** Continuous innovation and rapid iteration drive our mission to build smarter, ethical, and accessible AI.
- **Inclusivity & Ethics:** We prioritize responsible AI development that benefits society at large and encourages diverse participation.

## Careers & Join Us
Hugging Face is a dynamic workplace inviting passionate individuals to contribute to meaningful AI advancements. We seek creative thinkers and dedicated technologists eager to shape the future of technology and open-source collaboration.

## Who We Serve
Our users include individual developers, startups, academia, and Fortune 500 companies—all working together to harness AI's potential responsibly and effectively.

## Join Our Community
Become part of a global movement transforming possibilities into realities with Hugging Face. Explore our platform, participate in collaborative projects, and help build an open AI future accessible to everyone.

---

**Hugging Face** — Building the future of AI, today.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochuer(company_name,url):
    stream=client.chat.completions.create(model="gpt-4.1-nano",
    messages=[
        {"role":"system","content":brochure_system_prompt},
        {"role":"user","content":get_brochure_user_prompt(company_name,url)}
    ],
    stream=True)
    response=""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)


In [39]:
stream_brochuer("HuggingFace", "https://huggingface.co")

TypeError: Completions.create() got an unexpected keyword argument 'Stream'